# Neural Network from scratch.

Imports

In [210]:
import numpy as np
import random

Defining the sigmoid funciton.

In [211]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

Defining the Sigmoid function with +ve z.

In [212]:
def sigmoid_pos(z):
    return 1.0/(1.0+np.exp(z))

Defining derivative of Sigmoid function.

In [213]:
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

Defining derivative of Sigmoid function with +ve z.

In [214]:
def sigmoid_pos_prime(z):
    """Derivative of the sigmoid function."""
    return -(sigmoid_pos(z)*(1-sigmoid_pos(z)))

Create Network class for neural network.

In [215]:

class Network():

    def __init__(self, sizes, activation_function=sigmoid, activation_function_prime=sigmoid_prime):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        # Take the activation function as an argument and initialize it as a class method
        self.activation_function = activation_function
        self.activation_function_prime = activation_function_prime

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = self.activation_function(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs. If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            cost_arr = [] # Keep track of the cost for each epoch
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
                cost_arr.append(self.cost(mini_batch))
            if test_data:
                print(f"Epoch {j}: {self.evaluate(test_data)} / {n_test} Cost: {np.mean(cost_arr)}" )
            else:
                print(f"Epoch {j} complete")

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                        for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = self.activation_function(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            self.activation_function_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = self.activation_function_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost(self, mini_batch):
            """Return the cost of the network for each epoch"""
            square_error = [sum((y - self.feedforward(x))**2)
                            for x, y in mini_batch]
            return np.mean(square_error) / 2
                
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives (partial C_x) /
        (partial y) for the output activations."""
        return (output_activations-y)

Loading data using mnist_loader tool.

In [216]:
from tools import mnist_loader
# take only train_data and test_data
training_data, _, test_data = mnist_loader.load_data_wrapper()

Initializing network object with sigmoid activation function.

In [217]:
nn_with_sigmoid = Network(
                [784, 30, 10],activation_function=sigmoid,
                activation_function_prime=sigmoid_prime
                )

Training

In [218]:
nn_with_sigmoid.SGD(training_data, 10, 10, 3.0, test_data=test_data)

Epoch 0: 9027 / 10000 Cost: 0.16163291619409526
Epoch 1: 9272 / 10000 Cost: 0.04670256739717488
Epoch 2: 9297 / 10000 Cost: 0.037656464925146474
Epoch 3: 9373 / 10000 Cost: 0.033746583408457105
Epoch 4: 9364 / 10000 Cost: 0.031099051258455437
Epoch 5: 9413 / 10000 Cost: 0.02908603043214737
Epoch 6: 9451 / 10000 Cost: 0.027677727911419935
Epoch 7: 9448 / 10000 Cost: 0.026480499675919818
Epoch 8: 9465 / 10000 Cost: 0.025446269972815183
Epoch 9: 9464 / 10000 Cost: 0.024294467716664464


Initializing network object with sigmoid_pos activation function.

In [219]:
nn_with_sigmoid_pos = Network(
                [784, 30, 10],activation_function=sigmoid_pos,
                activation_function_prime=sigmoid_pos_prime
                )

Training.

In [220]:
nn_with_sigmoid_pos.SGD(training_data, 10, 10, 3.0, test_data=test_data)

Epoch 0: 9046 / 10000 Cost: 0.1050941873026213
Epoch 1: 9198 / 10000 Cost: 0.047060219845121654
Epoch 2: 9287 / 10000 Cost: 0.039039720245177455
Epoch 3: 9314 / 10000 Cost: 0.03480262992602772
Epoch 4: 9364 / 10000 Cost: 0.0320667141644793
Epoch 5: 9419 / 10000 Cost: 0.029754521818597647
Epoch 6: 9414 / 10000 Cost: 0.028039272477261984
Epoch 7: 9421 / 10000 Cost: 0.02692663811396091
Epoch 8: 9384 / 10000 Cost: 0.02561145437877834
Epoch 9: 9424 / 10000 Cost: 0.025269551380373283
